In [1]:
import numpy as np

Running the model requires pybamm to be installed on the "degradation" branch. The results are plotted in `figure_electrode_saturation_pybamm.py`.

## Solve the model

In [2]:
import pybamm

def lico2_volume_change_Ai2020(sto):
    omega = pybamm.Parameter("Positive electrode partial molar volume [m3.mol-1]")
    c_p_max = pybamm.Parameter("Maximum concentration in positive electrode [mol.m-3]")
    t_change = omega * c_p_max * sto
    return t_change

def graphite_volume_change_Ai2020(sto):
    p1 = 145.907
    p2 = -681.229
    p3 = 1334.442
    p4 = -1415.710
    p5 = 873.906
    p6 = -312.528
    p7 = 60.641
    p8 = -5.706
    p9 = 0.386
    p10 = -4.966e-05
    t_change = (
        p1 * sto ** 9
        + p2 * sto ** 8
        + p3 * sto ** 7
        + p4 * sto ** 6
        + p5 * sto ** 5
        + p6 * sto ** 4
        + p7 * sto ** 3
        + p8 * sto ** 2
        + p9 * sto
        + p10
    )
    return t_change

def get_parameter_values():
    parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
    parameter_values.update(
        {
            # mechanical properties
            "Positive electrode Poisson's ratio": 0.3,
            "Positive electrode Young's modulus [Pa]": 375e9,
            "Positive electrode reference concentration for free of deformation [mol.m-3]": 0,
            "Positive electrode partial molar volume [m3.mol-1]": -7.28e-7,
            "Positive electrode volume change": lico2_volume_change_Ai2020,
            # Loss of active materials (LAM) model
            "Positive electrode LAM constant exponential term": 2,
            "Positive electrode critical stress [Pa]": 375e6,
            # mechanical properties
            "Negative electrode Poisson's ratio": 0.2,
            "Negative electrode Young's modulus [Pa]": 15e9,
            "Negative electrode reference concentration for free of deformation [mol.m-3]": 0,
            "Negative electrode partial molar volume [m3.mol-1]": 3.1e-6,
            "Negative electrode volume change": graphite_volume_change_Ai2020,
            # Loss of active materials (LAM) model
            "Negative electrode LAM constant exponential term": 2,
            "Negative electrode critical stress [Pa]": 60e6,
            # Other
            "Cell thermal expansion coefficient [m.K-1]": 1.48E-6,
        },
        check_already_exists=False,
    )
    return parameter_values

parameter_values = get_parameter_values()
parameter_values.update(
    {
        "SEI kinetic rate constant [m.s-1]": 1e-15,
        "Positive electrode LAM constant propotional term": 1e-3,
        "Negative electrode LAM constant propotional term": 1e-3,
        "EC diffusivity [m2.s-1]": 2e-18,
    },
    check_already_exists=False,
)
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)

In [3]:
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = spm.param

Vmin = 3.0
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)
c_n_max = parameter_values.evaluate(param.c_n_max)
c_p_max = parameter_values.evaluate(param.c_p_max)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
)
print(esoh_sol["x_100"].data[0])
print(esoh_sol["y_100"].data[0])

0.8333742762029194
0.0335455473745956


In [4]:
eps_n = parameter_values["Negative electrode active material volume fraction"]
eps_p = parameter_values["Positive electrode active material volume fraction"]
C_over_eps_n = Cn / eps_n
C_over_eps_p = Cp / eps_p

In [5]:
pybamm.set_logging_level("NOTICE")
experiment = pybamm.Experiment(
    [
        ("Discharge at 1C until 3V",
         "Rest for 1 hour",
         "Charge at 1C until 4.2V", 
         "Hold at 4.2V until C/50")
    ] * 1200,
    termination="60% capacity",
#     cccv_handling="ode",
)

Parameter values that give a knee point

In [6]:
parameter_values_knee = get_parameter_values()
parameter_values_knee.update(
    {
        "SEI kinetic rate constant [m.s-1]": 1e-17,
        "Positive electrode LAM constant propotional term": 7e-3,
        "Negative electrode LAM constant propotional term": 7e-3,
        "EC diffusivity [m2.s-1]": 2e-18,
    },
    check_already_exists=False,
)

In [7]:
sim_long_knee = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values_knee, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long_knee = sim_long_knee.solve(initial_soc=1)

2021-08-20 16:51:01,354 - [NOTICE] simulation.solve(784): Cycle 1/1200 (36.252 ms elapsed) --------------------
2021-08-20 16:51:01,355 - [NOTICE] simulation.solve(818): Cycle 1/1200, step 1/4: Discharge at 1C until 3V
2021-08-20 16:51:01,466 - [NOTICE] simulation.solve(818): Cycle 1/1200, step 2/4: Rest for 1 hour
2021-08-20 16:51:01,521 - [NOTICE] simulation.solve(818): Cycle 1/1200, step 3/4: Charge at 1C until 4.2V
2021-08-20 16:51:01,606 - [NOTICE] simulation.solve(818): Cycle 1/1200, step 4/4: Hold at 4.2V until C/50
2021-08-20 16:51:01,897 - [NOTICE] simulation.solve(896): Capacity is now 4.968 Ah (originally 4.968 Ah, will stop at 2.981 Ah)
2021-08-20 16:51:01,897 - [NOTICE] simulation.solve(784): Cycle 2/1200 (579.417 ms elapsed) --------------------
2021-08-20 16:51:01,898 - [NOTICE] simulation.solve(818): Cycle 2/1200, step 1/4: Discharge at 1C until 3V
2021-08-20 16:51:01,936 - [NOTICE] simulation.solve(818): Cycle 2/1200, step 2/4: Rest for 1 hour
2021-08-20 16:51:01,965 -

## Export results to pickle file

In [18]:
sumvar = sol_long_knee.summary_variables

import pickle
filehandler = open("saturation.pkl","wb")
pickle.dump(sumvar, filehandler)